In [4]:
import cv2
import numpy as np
import time
import math

WIND_X = 1200
WIND_Y = 750

class Branch:
    def __init__(self, length_, angle_, color_, angle_range_,
                 length_decrement_factor_, max_depth_, branching_factor_):
        self.length = length_ #+ np.random.randint (int (length_ / 3)) - \
                             #int (length_ / 6)
        self.angle  = angle_ + float (np.random.randint (300) - 150) / 10000
        
        self.color  = color_
        
        if (max_depth_ < 1):
            self.color = (color_[0] + 10, color_[0] + 30, color_[0] + 20)
        
        self.children = []
        
        if (max_depth_ > 0):
            if (branching_factor_ > 1):
                angle_step = angle_range_ / (branching_factor_ - 1)
            
            else:
                angle_step = angle_range_ / 2
            
            for i in range (branching_factor_):
                self.add_child (length_ = int (self.length * length_decrement_factor_),
                                angle_  = - angle_range_ / 2 +
                                    self.angle + i * angle_step,
                                color_  = self.color,
                                angle_range_ = angle_range_,
                                length_decrement_factor_ = length_decrement_factor_,
                                max_depth_ = max_depth_ - 1,
                                branching_factor_ = branching_factor_)

    def add_child (self, length_, angle_, color_, angle_range_,
                   length_decrement_factor_, max_depth_, branching_factor_):
        new_branch = Branch (length_, angle_, color_, angle_range_,
                             length_decrement_factor_,
                             max_depth_, branching_factor_)
        
        self.children.append (new_branch)
    
    def draw (self, img, x, y, depth, tick):
        x1 = int (x + self.length * math.cos (self.angle))
        y1 = int (y + self.length * math.sin (self.angle))

        cv2.line (img, (x, y), (x1, y1), self.color, max(1, int(self.length / 65)))
        
        if (depth == 0):
            cv2.circle (img, (int (x1), int (y1)), 3, ((10 + tick * 7) % 255 + 10,
                        40, 10), -1)

        for child in self.children:
            child.draw (img, x1, y1, depth - 1, tick)

canvas_ = np.zeros ((WIND_Y, WIND_X, 3), np.uint8) * 75
canvas = canvas_.copy ()

tick = 0
shift = 4

cv2.namedWindow("render", cv2.WINDOW_AUTOSIZE)
cv2.namedWindow("trackbars", cv2.WINDOW_AUTOSIZE)

def nothing (smol_doge_argumemt__passed_but_not_umsed):
    pass

cv2.createTrackbar('length', 'trackbars', 170, 300, nothing)
cv2.createTrackbar('range', 'trackbars', 281, 628, nothing)
cv2.createTrackbar('decrement_factor', 'trackbars', 62, 200, nothing)
cv2.createTrackbar('depth', 'trackbars', 3, 7, nothing)
cv2.createTrackbar('branching', 'trackbars', 2, 7, nothing)
cv2.createTrackbar('red', 'trackbars', 50, 255, nothing)
cv2.createTrackbar('gre', 'trackbars', 120, 255, nothing)
cv2.createTrackbar('blu', 'trackbars', 190, 255, nothing)

while (True):
    tick += 1
    
    tree = Branch (length_ = cv2.getTrackbarPos('length', 'trackbars'),# + 70 * math.sin (tick *  0.02),
                   angle_ = -1.57,
                   #color_ = (10, 20, (10 + tick * 7) % 255),
                   color_ = (cv2.getTrackbarPos('blu', 'trackbars'),
                             cv2.getTrackbarPos('gre', 'trackbars'),
                             cv2.getTrackbarPos('red', 'trackbars')),
                   angle_range_ = cv2.getTrackbarPos('range', 'trackbars') / 100,
                   length_decrement_factor_ = cv2.getTrackbarPos('decrement_factor', 'trackbars') / 100,
                   max_depth_ = cv2.getTrackbarPos('depth', 'trackbars'),
                   branching_factor_ = cv2.getTrackbarPos('branching', 'trackbars'))
    
#     root = Branch (length_ = 140 + 30 * math.sin (tick *  0.05),
#                    angle_ = 1.57,
#                    #color_ = (10, 20, (10 + tick * 7) % 255),
#                    color_ = (20 % 255, 50, 90),
#                    angle_range_ = 2.81, length_decrement_factor_ = 0.62,
#                    max_depth_ = 3, branching_factor_ = 4)

    #if (tick > 0):
    canvas = canvas_.copy ()

    tree.draw (canvas, 600, 550, 2, tick)
    #root.draw (canvas, int (440), 700, 2, tick)
        
    shadow = canvas.astype ("float") * 0.95
    new_canvas = canvas_.copy ()

    new_canvas [:, shift :, :] = shadow [:, : -shift, :]

    canvas = new_canvas

    cv2.imshow ("render", canvas)

    time.sleep (0.05)
    
    key = cv2.waitKey (1) & 0xFF
    
    if (key == ord ('q')):
        break
    
cv2.destroyAllWindows ()